### Load the Oxford 102 Flowers Dataset

In [14]:
import torchvision.transforms as transforms
from torchvision.datasets import Flowers102
from torch.utils.data import DataLoader, random_split, ConcatDataset
import torch

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet mean
                         std=[0.229, 0.224, 0.225])   # ImageNet std
])

# Download Oxford Flowers102 dataset
# Load train, val, and test separately
train_data = Flowers102(root='./data', split='train', download=True, transform=transform)
val_data = Flowers102(root='./data', split='val', download=True, transform=transform)
test_data = Flowers102(root='./data', split='test', download=True, transform=transform)

# Combine into one full dataset (8189 images)
full_dataset = ConcatDataset([train_data, val_data, test_data])

# 70% train, 15% val, 15% test
train_size = int(0.7 * len(full_dataset))       # ~5732
val_size = int(0.15 * len(full_dataset))        # ~1228
test_size = len(full_dataset) - train_size - val_size  # ~1229

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)  # reproducibility
)

# Split into train/val
# train_size = int(0.8 * len(dataset))
# val_size = len(dataset) - train_size
# train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [15]:
len(train_dataset), len(val_dataset), len(test_dataset)

(5732, 1228, 1229)

### Load Pre-trained ResNet18 and Modify the final layer

In [16]:
import torch.nn as nn
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# Freeze feature extractor layers
for param in model.parameters():
    param.requires_grad = False

# Replace the final layer
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 102)  # 102 flower classes

model = model.to(device)


### Set Loss Function and Optimizer

In [17]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)  # only train final layer


### Train the Model

In [19]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        acc = 100 * correct / total
        print(f"Epoch {epoch+1}, Loss: {running_loss:.4f}, Validation Accuracy: {acc:.2f}%")

train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5)


Epoch 1, Loss: 21.4923, Validation Accuracy: 89.98%
Epoch 2, Loss: 20.0313, Validation Accuracy: 90.55%
Epoch 3, Loss: 18.0971, Validation Accuracy: 90.31%
Epoch 4, Loss: 16.8713, Validation Accuracy: 90.39%
Epoch 5, Loss: 15.9006, Validation Accuracy: 90.07%


### Evaluate on Test Set

In [20]:
test_loader = DataLoader(test_dataset, batch_size=32)
model.eval()
correct, total = 0, 0

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Test Accuracy: 91.38%


In [21]:
torch.save(model.state_dict(), "flower_classifier_resnet18.pth")